In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime

# --- Configuration ---
original_data_file = 'full_red_spider_count.csv'
target_total_rows = 10000
default_start_date_original = '1998-01-01'
output_file = 'red_spider_mite_forecast_data.csv'
random_seed = 42  # For reproducibility
np.random.seed(random_seed)

# --- Step 1: Load Original Data ---
try:
    df_original = pd.read_csv(original_data_file)
    print(f"Successfully loaded original data from: {original_data_file}")
    print(f"Original data shape (before index): {df_original.shape}")

    date_col_found = False
    date_col_options = ['Date', 'date', 'Timestamp', 'timestamp']
    for col in date_col_options:
        if col in df_original.columns:
            try:
                df_original[col] = pd.to_datetime(df_original[col])
                df_original = df_original.set_index(col)
                date_col_found = True
                print(f"Successfully set '{col}' as DatetimeIndex.")
                break
            except Exception as e:
                print(f"Warning: Could not convert column '{col}' to datetime. Error: {e}")

    if not date_col_found:
        print("\nWarning: No suitable Date column found. Assuming daily readings from '1998-01-01'.")
        num_original_rows = len(df_original)
        generated_dates = pd.date_range(start=default_start_date_original, periods=num_original_rows, freq='D')
        df_original.index = generated_dates
        print(f"Assigned DatetimeIndex from {generated_dates.min().strftime('%Y-%m-%d')} to {generated_dates.max().strftime('%Y-%m-%d')}.")

    required_cols = [
        'N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label',
        'humidity_sensor_temp', 'pressure', 'vapor_pressure', 'rain_bucket_capacity',
        'rain_bucket_tpm', 'fallen_rain_mm', 'wind_speed_mps', 'wind_direction',
        'wind_gust_mps', 'wind_north_mps', 'wind_east_mps', 'latitude', 'longitude',
        'x_orientation', 'y_orientation', 'air_temp'
    ]
    for col in required_cols:
        if col not in df_original.columns:
            print(f"Warning: Column '{col}' not found. Adding placeholder.")
            if col == 'label':
                df_original[col] = 'unknown'
            elif col in ['N', 'P', 'K']:
                df_original[col] = np.random.normal(50, 20, len(df_original)).astype(int)
            elif col == 'temperature' or col == 'air_temp':
                df_original[col] = np.random.normal(25, 5, len(df_original))
            elif col == 'humidity':
                df_original[col] = np.random.normal(70, 10, len(df_original))
            elif col == 'ph':
                df_original[col] = np.random.normal(6.5, 0.5, len(df_original))
            elif col == 'rainfall' or col == 'fallen_rain_mm':
                df_original[col] = np.random.exponential(scale=5.0, size=len(df_original))
            elif col == 'humidity_sensor_temp':
                df_original[col] = np.random.normal(22, 5, len(df_original))
            elif col == 'pressure':
                df_original[col] = np.random.normal(1013, 10, len(df_original))
            elif col == 'vapor_pressure':
                df_original[col] = np.random.normal(1, 0.5, len(df_original))
            elif col == 'rain_bucket_capacity':
                df_original[col] = 5
            elif col == 'rain_bucket_tpm':
                df_original[col] = 0
            elif col == 'wind_speed_mps':
                df_original[col] = np.random.exponential(scale=2.0, size=len(df_original))
            elif col == 'wind_direction':
                df_original[col] = np.random.uniform(0, 360, len(df_original))
            elif col == 'wind_gust_mps':
                df_original[col] = df_original['wind_speed_mps'] * 1.5
            elif col in ['wind_north_mps', 'wind_east_mps']:
                df_original[col] = 0
            elif col == 'latitude':
                df_original[col] = 38.6309375
            elif col == 'longitude':
                df_original[col] = -90.22617187
            elif col in ['x_orientation', 'y_orientation']:
                df_original[col] = 0
            else:
                df_original[col] = np.nan

    df_original = df_original[required_cols]
    df_original = df_original.sort_index()

    if df_original.isnull().any().any():
        print("\nWarning: NaNs in original data. Filling with ffill/bfill.")
        df_original.fillna(method='ffill', inplace=True)
        df_original.fillna(method='bfill', inplace=True)
        for col in df_original.columns:
            if df_original[col].isnull().any():
                df_original[col].fillna(df_original[col].mean() if pd.api.types.is_numeric_dtype(df_original[col]) else 'unknown', inplace=True)

    original_rows = len(df_original)
    last_original_date = df_original.index.max()
    start_synthetic_date = last_original_date + pd.Timedelta(days=1)
    print(f"\nLast date in original data: {last_original_date.strftime('%Y-%m-%d')}")
    print(f"Start date for synthetic data: {start_synthetic_date.strftime('%Y-%m-%d')}")

except FileNotFoundError:
    print(f"Error: File '{original_data_file}' not found. Generating synthetic data only.")
    required_cols = [
        'N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label',
        'humidity_sensor_temp', 'pressure', 'vapor_pressure', 'rain_bucket_capacity',
        'rain_bucket_tpm', 'fallen_rain_mm', 'wind_speed_mps', 'wind_direction',
        'wind_gust_mps', 'wind_north_mps', 'wind_east_mps', 'latitude', 'longitude',
        'x_orientation', 'y_orientation', 'air_temp'
    ]
    df_original = pd.DataFrame(columns=required_cols)
    original_rows = 0
    start_synthetic_date = pd.to_datetime(default_start_date_original)

# --- Step 2: Generate Synthetic Data ---
rows_to_generate = max(0, target_total_rows - original_rows)
if rows_to_generate > 0:
    print(f"\nGenerating {rows_to_generate} synthetic rows...")
    synthetic_dates = pd.date_range(start=start_synthetic_date, periods=rows_to_generate, freq='D')
    synthetic_day_of_year = synthetic_dates.dayofyear.to_numpy()
    synthetic_year = synthetic_dates.year.to_numpy() - synthetic_dates.year.min() + 1

    # Add long-term climate trends (slight warming over time)
    year_effect = (synthetic_year - 1) * 0.03

    # Temperature (air_temp and temperature synced)
    temp_base = 25
    temp_amplitude = 10
    seasonal_effect = np.sin((synthetic_day_of_year - 90) * 2 * np.pi / 365)
    multi_year_cycle = np.sin((synthetic_year - synthetic_year.min()) * 2 * np.pi / 4)
    synth_temperature = (
        temp_base +
        temp_amplitude * seasonal_effect +
        year_effect +
        multi_year_cycle * 1.5 +
        np.random.normal(0, 2.0, rows_to_generate)
    )
    synth_temperature = np.clip(synth_temperature, 8, 45)

    # Humidity
    humidity_base = 70
    humidity_amplitude = 15
    synth_humidity = (
        humidity_base -
        humidity_amplitude * seasonal_effect +
        np.random.normal(0, 7, rows_to_generate) -
        (synth_temperature - synth_temperature.mean()) * 0.5
    )
    synth_humidity = np.clip(synth_humidity, 20, 100)

    # Rainfall (fallen_rain_mm and rainfall synced)
    rain_seasonal = 0.2 + 0.3 * np.sin((synthetic_day_of_year - 150) * 2 * np.pi / 365)
    rain_chance = rain_seasonal + 0.05 * multi_year_cycle
    rain_indices = np.random.rand(rows_to_generate) < rain_chance
    rain_scale = 8.0 + 4.0 * rain_seasonal
    rain_amounts = np.random.exponential(scale=rain_scale, size=rows_to_generate)
    extreme_indices = (np.random.rand(rows_to_generate) < 0.01) & rain_indices
    rain_amounts[extreme_indices] *= np.random.uniform(3, 8, size=extreme_indices.sum())
    synth_rainfall = np.zeros(rows_to_generate)
    synth_rainfall[rain_indices] = rain_amounts[rain_indices]
    synth_rainfall = np.clip(synth_rainfall, 0, 956)
    rain_binary = (synth_rainfall > 0).astype(int)
    for i in range(1, len(rain_binary)):
        if rain_binary[i-1] == 0 and np.random.rand() < 0.8:
            rain_binary[i] = 0
    synth_rainfall = synth_rainfall * rain_binary

    # Humidity sensor temperature
    synth_humidity_sensor_temp = synth_temperature + np.random.normal(0, 1, rows_to_generate)
    synth_humidity_sensor_temp = np.clip(synth_humidity_sensor_temp, 5, 50)

    # Pressure
    pressure_base = 1013
    synth_pressure = (
        pressure_base -
        (synth_temperature - temp_base) * 0.5 -
        (synth_humidity - humidity_base) * 0.2 +
        np.random.normal(0, 5, rows_to_generate)
    )
    synth_pressure = np.clip(synth_pressure, 980, 1040)

    # Vapor pressure (simplified Clausius-Clapeyron relation)
    es = 6.1078 * 10 ** (7.5 * synth_temperature / (synth_temperature + 237.3))
    synth_vapor_pressure = (synth_humidity / 100) * es / 10
    synth_vapor_pressure += np.random.normal(0, 0.1, rows_to_generate)
    synth_vapor_pressure = np.clip(synth_vapor_pressure, 0, 5)

    # Rain bucket capacity (constant)
    synth_rain_bucket_capacity = np.full(rows_to_generate, 5.0)

    # Rain bucket tips per minute
    synth_rain_bucket_tpm = (synth_rainfall / synth_rain_bucket_capacity).astype(int)
    synth_rain_bucket_tpm = np.clip(synth_rain_bucket_tpm, 0, 10)

    # Wind speed
    synth_wind_speed = np.random.exponential(scale=2.0, size=rows_to_generate)
    synth_wind_speed = np.clip(synth_wind_speed, 0, 20)

    # Wind direction
    synth_wind_direction = np.zeros(rows_to_generate)
    synth_wind_direction[0] = 166
    for i in range(1, rows_to_generate):
        change = np.random.normal(0, 30)
        synth_wind_direction[i] = (synth_wind_direction[i-1] + change) % 360

    # Wind gusts
    synth_wind_gust = synth_wind_speed * np.random.uniform(1.2, 2.0, rows_to_generate)
    synth_wind_gust = np.clip(synth_wind_gust, synth_wind_speed, 25)

    # Wind components
    wind_radians = np.radians(synth_wind_direction)
    synth_wind_north = -synth_wind_speed * np.cos(wind_radians)
    synth_wind_east = synth_wind_speed * np.sin(wind_radians)

    # Latitude and longitude (constant)
    synth_latitude = np.full(rows_to_generate, 38.6309375)
    synth_longitude = np.full(rows_to_generate, -90.22617187)

    # Orientation
    synth_x_orientation = np.random.normal(-1, 0.1, rows_to_generate)
    synth_y_orientation = np.random.normal(0, 0.1, rows_to_generate)

    # Soil nutrients and pH
    N_base = df_original['N'].mean() if original_rows > 0 else 50
    P_base = df_original['P'].mean() if original_rows > 0 else 53
    K_base = df_original['K'].mean() if original_rows > 0 else 48
    ph_base = df_original['ph'].mean() if original_rows > 0 else 6.5
    synth_N = np.clip(N_base - 5 * np.sin(synthetic_day_of_year * 2 * np.pi / 365) + synth_rainfall * 0.1 + np.random.normal(0, 15, rows_to_generate), 5, 140).astype(int)
    synth_P = np.clip(P_base + 3 * np.sin((synthetic_day_of_year + 60) * 2 * np.pi / 365) - synth_rainfall * 0.05 + np.random.normal(0, 12, rows_to_generate), 5, 145).astype(int)
    synth_K = np.clip(K_base + 4 * np.sin((synthetic_day_of_year + 120) * 2 * np.pi / 365) + np.random.normal(0, 18, rows_to_generate), 5, 205).astype(int)
    synth_ph = np.clip(ph_base - 0.3 * np.sin(synthetic_day_of_year * 2 * np.pi / 365) - synth_rainfall * 0.005 + np.random.normal(0, 0.4, rows_to_generate), 3.5, 9.5)

    # Label
    if original_rows > 0 and df_original['label'].nunique() == 1:
        synth_label_value = df_original['label'].iloc[0]
    else:
        crop_types = ['tomato', 'bean', 'cucumber', 'strawberry', 'rose']
        crop_probs = [0.3, 0.2, 0.2, 0.15, 0.15]
        synth_label = np.random.choice(crop_types, size=rows_to_generate, p=crop_probs)

    df_synthetic = pd.DataFrame({
        'N': synth_N,
        'P': synth_P,
        'K': synth_K,
        'temperature': synth_temperature,
        'humidity': synth_humidity,
        'ph': synth_ph,
        'rainfall': synth_rainfall,
        'label': synth_label if 'synth_label' in locals() else [synth_label_value] * rows_to_generate,
        'humidity_sensor_temp': synth_humidity_sensor_temp,
        'pressure': synth_pressure,
        'vapor_pressure': synth_vapor_pressure,
        'rain_bucket_capacity': synth_rain_bucket_capacity,
        'rain_bucket_tpm': synth_rain_bucket_tpm,
        'fallen_rain_mm': synth_rainfall,
        'wind_speed_mps': synth_wind_speed,
        'wind_direction': synth_wind_direction,
        'wind_gust_mps': synth_wind_gust,
        'wind_north_mps': synth_wind_north,
        'wind_east_mps': synth_wind_east,
        'latitude': synth_latitude,
        'longitude': synth_longitude,
        'x_orientation': synth_x_orientation,
        'y_orientation': synth_y_orientation,
        'air_temp': synth_temperature
    }, index=synthetic_dates)

    print(f"Synthetic data generated. Shape: {df_synthetic.shape}")
else:
    df_synthetic = pd.DataFrame()

# --- Step 3: Combine Data ---
df_combined = pd.concat([df_original, df_synthetic], axis=0).sort_index()
print(f"\nCombined data shape: {df_combined.shape}")

if df_combined.isnull().any().any():
    print("\nWarning: NaNs in combined data. Filling with ffill/bfill.")
    df_combined.fillna(method='ffill', inplace=True)
    df_combined.fillna(method='bfill', inplace=True)

# --- Step 4: Red Spider Mite Simulation with Enhanced Dynamics ---
print("\nRunning enhanced Red Spider Mite simulation...")

initial_count = 10.0
temp_low = 15.0
temp_opt_low = 25.0
temp_opt_high = 35.0
temp_high = 40.0
humid_low = 35.0
humid_med = 60.0
humid_high = 70.0
humid_extreme = 90.0
rain_med = 5.0
rain_high = 10.0
base_survival = 0.97
sensitivity_divisor = 18.0
MAX_MITES_PER_LEAF = 600
predator_base = 0.08
predator_seasonal = 0.04
predator_response_lag = 14
pesticide_effect_duration = 14
pesticide_resistance_rate = 0.005
pesticide_threshold = 100
pesticide_effectiveness = 0.85
pesticide_decay_rate = 0.1
resistance_level = 0.0

df_combined['Red_spider_mite_count'] = 0.0
df_combined['Red_spider_mite_category'] = 'Low'
df_combined['Pesticide_active'] = 0
df_combined['Resistance_level'] = 0.0
df_combined['Predation_pressure'] = 0.0

df_combined.iloc[0, df_combined.columns.get_loc('Red_spider_mite_count')] = initial_count
df_combined.iloc[0, df_combined.columns.get_loc('Predation_pressure')] = predator_base

outbreak_points = np.random.choice(range(100, len(df_combined)-1000),
                                   size=np.random.randint(5, 10),
                                   replace=False)
intervention_points = []

days_since_treatment = 999
for i in range(1, len(df_combined)):
    previous_count = df_combined.iloc[i-1]['Red_spider_mite_count']
    temp = df_combined.iloc[i]['temperature']
    humid = df_combined.iloc[i]['humidity']
    rain = df_combined.iloc[i]['rainfall']
    wind = df_combined.iloc[i]['wind_speed_mps']
    current_resistance = df_combined.iloc[i-1]['Resistance_level']

    if days_since_treatment < pesticide_effect_duration:
        days_since_treatment += 1
        df_combined.iloc[i, df_combined.columns.get_loc('Pesticide_active')] = max(0,
            pesticide_effectiveness * (1 - days_since_treatment * pesticide_decay_rate) * (1 - current_resistance))
    else:
        df_combined.iloc[i, df_combined.columns.get_loc('Pesticide_active')] = 0

    if previous_count > pesticide_threshold or i in intervention_points:
        days_since_treatment = 0
        df_combined.iloc[i, df_combined.columns.get_loc('Pesticide_active')] = pesticide_effectiveness * (1 - current_resistance)
        intervention_points.append(i)
        new_resistance = current_resistance + pesticide_resistance_rate * (previous_count / MAX_MITES_PER_LEAF)
        df_combined.iloc[i, df_combined.columns.get_loc('Resistance_level')] = min(0.85, new_resistance)
    else:
        df_combined.iloc[i, df_combined.columns.get_loc('Resistance_level')] = max(0, current_resistance - 0.001)

    if i > predator_response_lag:
        avg_past_density = df_combined.iloc[i-predator_response_lag:i]['Red_spider_mite_count'].mean() / MAX_MITES_PER_LEAF
        seasonal_factor = 1 + predator_seasonal * np.sin((df_combined.index[i].dayofyear - 120) * 2 * np.pi / 365)
        predation = predator_base * seasonal_factor * (1 + 2 * avg_past_density)
        df_combined.iloc[i, df_combined.columns.get_loc('Predation_pressure')] = predation
    else:
        df_combined.iloc[i, df_combined.columns.get_loc('Predation_pressure')] = predator_base

    change_factor = 0.0
    if temp > temp_high:
        change_factor -= 2.0 * ((temp - temp_high) / 5)
    elif temp_opt_low <= temp <= temp_opt_high:
        temp_optimality = 1 - abs(temp - 32) / 7
        change_factor += 1.8 * temp_optimality
    elif temp < temp_low:
        change_factor -= 1.2 * ((temp_low - temp) / 5)

    if humid < humid_low:
        change_factor += 1.2
    elif humid < humid_med:
        change_factor += 0.7 * (humid_med - humid) / (humid_med - humid_low)
    elif humid > humid_extreme:
        change_factor -= 1.8
    elif humid > humid_high:
        change_factor -= 1.0 * (humid - humid_high) / (humid_extreme - humid_high)

    if i in outbreak_points:
        previous_count = max(previous_count, np.random.randint(30, 50))
        change_factor += np.random.uniform(1.0, 2.0)
        print(f"Outbreak seed injected at day {i} (date: {df_combined.index[i].strftime('%Y-%m-%d')})")

    r = change_factor / sensitivity_divisor
    density_factor = 1 - (previous_count / MAX_MITES_PER_LEAF) ** 1.2
    growth_rate = 1.0 + r * density_factor

    pesticide_effect = df_combined.iloc[i]['Pesticide_active']
    predation = df_combined.iloc[i]['Predation_pressure'] * previous_count / 100

    potential_new_count = (
        previous_count * growth_rate * base_survival -
        predation -
        (previous_count * pesticide_effect)
    )

    noise_level = max(1.0, np.sqrt(previous_count) / 3)
    noise = np.random.normal(0, noise_level)
    potential_new_count = max(0.0, potential_new_count + noise)

    if rain > rain_high:
        wash_off = 0.4 + 0.1 * min(1.0, (rain - rain_high) / 20)
        potential_new_count *= (1 - wash_off)
    elif rain > rain_med:
        potential_new_count *= 0.85

    # Add wind effect
    if wind > 10:
        potential_new_count *= (1 - min(0.3, (wind - 10) / 20))

    new_count = max(0.1, min(potential_new_count, MAX_MITES_PER_LEAF))
    df_combined.iloc[i, df_combined.columns.get_loc('Red_spider_mite_count')] = new_count

df_combined['Red_spider_mite_count'] = df_combined['Red_spider_mite_count'].round().astype(int)

def assign_category(count):
    if count <= 10: return 'Low'
    elif count <= 50: return 'Medium'
    elif count <= 200: return 'High'
    else: return 'Severe'

df_combined['Red_spider_mite_category'] = df_combined['Red_spider_mite_count'].apply(assign_category)

print("\nSimulation complete. Distribution:")
print(df_combined['Red_spider_mite_category'].value_counts())
print("\nMite Count Statistics:")
print(df_combined['Red_spider_mite_count'].describe())

plt.figure(figsize=(15, 8))
plt.plot(df_combined.index, df_combined['Red_spider_mite_count'])
plt.title('Red Spider Mite Population Over Time')
plt.ylabel('Count')
plt.xlabel('Date')
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()

try:
    plt.savefig('mite_population_timeseries.png')
    print("Saved plot to mite_population_timeseries.png")
except Exception as e:
    print(f"Could not save plot: {e}")
plt.close()

# --- Step 5: Enhanced Feature Engineering ---
print("\nApplying enhanced feature engineering...")

lags = [1, 3, 7, 14, 21, 30]
features = [
    'Red_spider_mite_count', 'temperature', 'humidity', 'rainfall', 'Pesticide_active',
    'Resistance_level', 'wind_speed_mps', 'pressure', 'vapor_pressure'
]

for feature in features:
    for lag in lags:
        df_combined[f'{feature}_lag_{lag}'] = df_combined[feature].shift(lag)

windows = [3, 7, 14, 30, 60]
for window in windows:
    df_combined[f'count_roll_mean_{window}'] = df_combined['Red_spider_mite_count'].rolling(window=window, min_periods=1).mean()
    df_combined[f'count_roll_max_{window}'] = df_combined['Red_spider_mite_count'].rolling(window=window, min_periods=1).max()
    df_combined[f'count_roll_min_{window}'] = df_combined['Red_spider_mite_count'].rolling(window=window, min_periods=1).min()
    df_combined[f'count_roll_std_{window}'] = df_combined['Red_spider_mite_count'].rolling(window=window, min_periods=1).std().fillna(0)
    df_combined[f'count_growth_rate_{window}'] = (
        df_combined['Red_spider_mite_count'] / (df_combined[f'count_roll_mean_{window}'].shift(1) + 0.1) - 1
    )

    for feature in ['temperature', 'humidity', 'rainfall', 'wind_speed_mps', 'pressure', 'vapor_pressure']:
        df_combined[f'{feature}_roll_mean_{window}'] = df_combined[feature].rolling(window=window, min_periods=1).mean()
        df_combined[f'{feature}_roll_std_{window}'] = df_combined[feature].rolling(window=window, min_periods=1).std().fillna(0)

        if feature == 'temperature':
            df_combined[f'{feature}_roll_max_{window}'] = df_combined[feature].rolling(window=window, min_periods=1).max()
            df_combined[f'{feature}_roll_min_{window}'] = df_combined[feature].rolling(window=window, min_periods=1).min()
            df_combined[f'{feature}_extrema_{window}'] = df_combined[f'{feature}_roll_max_{window}'] - df_combined[f'{feature}_roll_min_{window}']
        elif feature == 'humidity':
            df_combined[f'{feature}_roll_max_{window}'] = df_combined[feature].rolling(window=window, min_periods=1).max()
            df_combined[f'{feature}_roll_min_{window}'] = df_combined[feature].rolling(window=window, min_periods=1).min()
        elif feature == 'rainfall':
            df_combined[f'{feature}_roll_sum_{window}'] = df_combined[feature].rolling(window=window, min_periods=1).sum()
            df_combined[f'{feature}_roll_max_{window}'] = df_combined[feature].rolling(window=window, min_periods=1).max()
            df_combined[f'rainy_days_{window}'] = df_combined['rainfall'].apply(lambda x: 1 if x > 0 else 0).rolling(window=window, min_periods=1).sum()

df_combined['month'] = df_combined.index.month
df_combined['day_of_year'] = df_combined.index.dayofyear
df_combined['day_of_week'] = df_combined.index.dayofweek
df_combined['week_of_year'] = df_combined.index.isocalendar().week.astype(int)
df_combined['quarter'] = df_combined.index.quarter
df_combined['year'] = df_combined.index.year
df_combined['days_from_start'] = (df_combined.index - df_combined.index.min()).days

df_combined['month_sin'] = np.sin(2 * np.pi * df_combined['month'] / 12)
df_combined['month_cos'] = np.cos(2 * np.pi * df_combined['month'] / 12)
df_combined['day_of_year_sin'] = np.sin(2 * np.pi * df_combined['day_of_year'] / 365.25)
df_combined['day_of_year_cos'] = np.cos(2 * np.pi * df_combined['day_of_year'] / 365.25)

forecast_horizons = [1, 7, 14, 30]
for horizon in forecast_horizons:
    df_combined[f'target_count_{horizon}d'] = df_combined['Red_spider_mite_count'].shift(-horizon)
    threshold = 50
    df_combined[f'target_outbreak_{horizon}d'] = (df_combined['Red_spider_mite_count'].shift(-horizon) > threshold).astype(int)
    df_combined[f'target_category_{horizon}d'] = df_combined['Red_spider_mite_category'].shift(-horizon)

df_combined['temp_humid_interaction'] = df_combined['temperature'] * df_combined['humidity'] / 100
df_combined['rain_temp_interaction'] = df_combined['rainfall'] * df_combined['temperature'] / 10
df_combined['optimal_growth_condition'] = (
    (df_combined['temperature'] >= temp_opt_low) &
    (df_combined['temperature'] <= temp_opt_high) &
    (df_combined['humidity'] < humid_high)
).astype(int)
df_combined['recent_pesticide'] = (df_combined['Pesticide_active'] > 0).astype(int)

# --- Step 6: Handle Missing Values and Split Data ---
df_final = df_combined.copy()
max_lag = max(lags)
min_required_rows = max_lag + max(forecast_horizons)
df_final = df_final.iloc[min_required_rows:]

if df_final.isnull().any().any():
    print("\nFilling remaining NaN values...")
    for col in df_final.columns:
        if df_final[col].isnull().any():
            if pd.api.types.is_numeric_dtype(df_final[col]):
                df_final[col].fillna(df_final[col].mean(), inplace=True)
            else:
                df_final[col].fillna(df_final[col].mode()[0], inplace=True)

try:
    df_final.to_csv(output_file)
    print(f"\nData saved to {output_file}")
    print(f"Total rows in final dataset: {len(df_final)}")
    print(f"Total features generated: {len(df_final.columns)}")

    print("\nTarget variable distributions:")
    for horizon in forecast_horizons:
        outbreak_col = f'target_outbreak_{horizon}d'
        count_col = f'target_count_{horizon}d'
        if outbreak_col in df_final.columns:
            outbreak_rate = df_final[outbreak_col].mean() * 100
            print(f"Outbreak rate ({horizon}-day forecast): {outbreak_rate:.2f}%")
        if count_col in df_final.columns:
            print(f"Count distribution ({horizon}-day forecast):")
            print(df_final[count_col].describe().round(1))

    horizon_to_plot = 7
    fig, axs = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
    sample_size = min(500, len(df_final))
    sample_start = max(0, len(df_final) - sample_size)
    sample_data = df_final.iloc[sample_start:sample_start+sample_size]

    axs[0].plot(sample_data.index, sample_data['Red_spider_mite_count'],
                label='Current Count', color='blue')
    axs[0].plot(sample_data.index, sample_data[f'target_count_{horizon_to_plot}d'],
                label=f'{horizon_to_plot}-Day Future Count', color='red', linestyle='--')
    axs[0].set_title(f'Mite Count: Current vs {horizon_to_plot}-Day Future')
    axs[0].set_ylabel('Count')
    axs[0].legend()
    axs[0].grid(True, alpha=0.3)

    axs[1].plot(sample_data.index, sample_data['temperature'],
                label='Temperature (°C)', color='orange')
    axs[1].plot(sample_data.index, sample_data['humidity'],
                label='Humidity (%)', color='blue')
    axs[1].set_ylabel('Value')
    axs[1].legend()
    axs[1].grid(True, alpha=0.3)

    plt.tight_layout()
    try:
        plt.savefig('timeseries_forecast_preview.png')
        print("Saved forecast preview plot to timeseries_forecast_preview.png")
    except Exception as e:
        print(f"Could not save forecast preview plot: {e}")
    plt.close()

    print("\nDataset ready for time-series forecasting.")
    print("\nSample rows from the final dataset:")
    try:
        print(df_final.iloc[[0, len(df_final)//2, -1], :10])
    except:
        print("Could not display sample rows.")

    print("\nFeature categories generated for forecasting:")
    print("1. Base features: Environmental conditions, soil metrics")
    print("2. Lag features: Historical values of key metrics")
    print("3. Rolling statistics: Trends and variability over different windows")
    print("4. Seasonal features: Calendar-based and cyclical patterns")
    print("5. Interaction features: Combined effects of multiple variables")
    print("6. Management features: Pesticide application and resistance metrics")
    print("7. Target variables: Future mite counts and outbreak indicators")

    print("\nDataset successfully prepared for various forecasting approaches:")
    print("- Regression models for predicting exact mite counts")
    print("- Classification models for predicting outbreak risk")
    print("- Time series models for capturing sequential patterns")
    print("- Anomaly detection for identifying unusual patterns")
except Exception as e:
    print(f"Error saving data: {e}")

Successfully loaded original data from: full_red_spider_count.csv
Original data shape (before index): (2200, 8)

Assigned DatetimeIndex from 1998-01-01 to 2004-01-09.

Last date in original data: 2004-01-09
Start date for synthetic data: 2004-01-10

Generating 7800 synthetic rows...
Synthetic data generated. Shape: (7800, 24)

Combined data shape: (10000, 24)

Running enhanced Red Spider Mite simulation...
Outbreak seed injected at day 913 (date: 2000-07-02)
Outbreak seed injected at day 1397 (date: 2001-10-29)
Outbreak seed injected at day 1989 (date: 2003-06-13)
Outbreak seed injected at day 2858 (date: 2005-10-29)
Outbreak seed injected at day 6308 (date: 2015-04-10)
Outbreak seed injected at day 6553 (date: 2015-12-11)
Outbreak seed injected at day 6973 (date: 2017-02-03)
Outbreak seed injected at day 7237 (date: 2017-10-25)

Simulation complete. Distribution:
Low       8710
Medium    1062
High       228
Name: Red_spider_mite_category, dtype: int64

Mite Count Statistics:
count    

/var/folders/_2/7yt2d7hn1ybcynjw9_72nftw0000gn/T/ipykernel_12276/2788964740.py:477: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined[f'{feature}_roll_min_{window}'] = df_combined[feature].rolling(window=window, min_periods=1).min()
/var/folders/_2/7yt2d7hn1ybcynjw9_72nftw0000gn/T/ipykernel_12276/2788964740.py:468: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined[f'{feature}_roll_mean_{window}'] = df_combined[feature].rolling(window=window, min_periods=1).mean()
/var/folders/_2/7yt2d7hn1ybcynjw9_72nftw0000gn/T/i


Data saved to red_spider_mite_forecast_data.csv
Total rows in final dataset: 9940
Total features generated: 235

Target variable distributions:
Outbreak rate (1-day forecast): 2.29%
Count distribution (1-day forecast):
count    9940.0
mean        5.9
std        12.8
min         0.0
25%         0.0
50%         2.0
75%         5.0
max       106.0
Name: target_count_1d, dtype: float64
Outbreak rate (7-day forecast): 2.29%
Count distribution (7-day forecast):
count    9940.0
mean        5.9
std        12.8
min         0.0
25%         0.0
50%         2.0
75%         5.0
max       106.0
Name: target_count_7d, dtype: float64
Outbreak rate (14-day forecast): 2.29%
Count distribution (14-day forecast):
count    9940.0
mean        5.9
std        12.8
min         0.0
25%         0.0
50%         2.0
75%         5.0
max       106.0
Name: target_count_14d, dtype: float64
Outbreak rate (30-day forecast): 2.29%
Count distribution (30-day forecast):
count    9940.0
mean        5.9
std        12.8
min 

In [4]:
df_final.columns

Index(['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label',
       'humidity_sensor_temp', 'pressure',
       ...
       'target_count_14d', 'target_outbreak_14d', 'target_category_14d',
       'target_count_30d', 'target_outbreak_30d', 'target_category_30d',
       'temp_humid_interaction', 'rain_temp_interaction',
       'optimal_growth_condition', 'recent_pesticide'],
      dtype='object', length=235)